In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

C:\Users\hubak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/556181
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
feature_view = fs.get_feature_view(
    name='bitcoin_training_fv',
    version=1
)

In [3]:
import joblib
model = mr.get_model(
    name="bitcoin_price_prediction_model", 
    version=1
)

saved_model_dir = model.download()

xgboost_model = joblib.load(saved_model_dir + "/bitcoin_price_prediction_model.pkl")

In [4]:
bitcoin_fg = fs.get_feature_group(
    name='bitcoin_price',
    version=2,
)

In [5]:
data = bitcoin_fg.select_all()
version = 1 
feature_view = fs.get_or_create_feature_view(
    name='bitcoin_training_fv',
    version=version,
    query=data
)

In [6]:
df = feature_view.get_batch_data()

Finished: Reading data from Hopsworks, using ArrowFlight (1.12s) 


In [7]:
sorted_df = df.sort_values(by='timestamp')

In [8]:
sorted_df.tail()

,timestamp,date,open,high,low,volume,close,tomorrow
1130,1714514400000,2024-05-01,60609.496094,60780.500000,56555.292969,4.843978e+10,58254.011719,59123.433594
1307,1714600800000,2024-05-02,58253.703125,59602.296875,56937.203125,3.271181e+10,59123.433594,62889.835938
2583,1714687200000,2024-05-03,59122.300781,63320.503906,58848.312500,3.317202e+10,62889.835938,63891.472656
1454,1714773600000,2024-05-04,62891.031250,64494.957031,62599.351562,2.062048e+10,63891.472656,64031.132812
1109,1714860000000,2024-05-05,63892.453125,64610.890625,62955.304688,1.829616e+10,64031.132812,NaN


In [8]:
X = sorted_df.drop(columns=['timestamp', 'date', 'tomorrow'])

In [9]:
day_to_predict = X.tail(1).iloc[0]
day_to_predict_reshaped = day_to_predict.values.reshape(1, -1)

In [10]:
pred = xgboost_model.predict(day_to_predict_reshaped)

In [12]:
import pandas as pd

prediction_df = pd.DataFrame({'Date of last available closing price': sorted_df['date'].iloc[-1], 'Predicted value for the next day': pred})

In [13]:
prediction_df

,Date of last available closing price,Predicted value for the next day
0,2024-05-05,65527.914062
